In [ ]:
pip install vaderSentiment

In [2]:
import pandas as pd
import re
import datetime
import matplotlib.pylab as plt
%matplotlib inline

In [3]:
df_btc = pd.read_csv('btc-usdt.csv')

In [4]:
df_btc

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,TB Base Volume,TB Quote Volume,Ignore
0,2019-07-30 00:00:00.000000000,9509.07,9535.00,9503.07,9517.75,381.293856,2019-07-30 00:29:59.999000064,3.631211e+06,3492,216.528742,2.062189e+06,0
1,2019-07-30 00:30:00.000000000,9517.03,9539.00,9507.00,9532.58,258.514869,2019-07-30 00:59:59.999000064,2.462134e+06,3133,141.887550,1.351182e+06,0
2,2019-07-30 01:00:00.000000000,9533.59,9534.00,9500.00,9500.25,275.797270,2019-07-30 01:29:59.999000064,2.624322e+06,3583,114.238457,1.087402e+06,0
3,2019-07-30 01:30:00.000000000,9500.25,9514.46,9460.71,9465.25,485.218470,2019-07-30 01:59:59.999000064,4.604004e+06,5330,225.681291,2.141459e+06,0
4,2019-07-30 02:00:00.000000000,9465.25,9488.94,9430.01,9460.42,498.669626,2019-07-30 02:29:59.999000064,4.717269e+06,6197,228.535910,2.162163e+06,0
...,...,...,...,...,...,...,...,...,...,...,...,...
51306,2022-07-04 20:30:00.000000000,19783.48,19812.19,19747.34,19782.79,692.590040,2022-07-04 20:59:59.999000064,1.369969e+07,13963,374.688800,7.410945e+06,0
51307,2022-07-04 21:00:00.000000000,19782.79,19823.97,19749.51,19781.71,637.498450,2022-07-04 21:29:59.999000064,1.261613e+07,11000,325.724110,6.446203e+06,0
51308,2022-07-04 21:30:00.000000000,19781.71,20031.58,19777.66,19958.75,1298.124410,2022-07-04 21:59:59.999000064,2.587612e+07,26022,763.196040,1.521461e+07,0
51309,2022-07-04 22:00:00.000000000,19958.76,19981.87,19871.88,19956.60,1260.138300,2022-07-04 22:29:59.999000064,2.509992e+07,19054,636.915190,1.268486e+07,0


In [5]:
df_btc['Open Time'] = pd.to_datetime(df_btc['Open Time'])

In [6]:
df_btc['Close Time'] = pd.to_datetime(df_btc['Close Time'])

In [7]:
#Read and store our Bitcoin tweets csv to a variable
df_tweets = pd.read_csv("Bitcoin_tweets.csv")

c:\Users\Mo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (4,5,6,7,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3840626 entries, 0 to 3840625
Data columns (total 13 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_name         object
 1   user_location     object
 2   user_description  object
 3   user_created      object
 4   user_followers    object
 5   user_friends      object
 6   user_favourites   object
 7   user_verified     object
 8   date              object
 9   text              object
 10  hashtags          object
 11  source            object
 12  is_retweet        object
dtypes: object(13)
memory usage: 380.9+ MB


In [9]:
#Drop any row where the "text" column is NaN
df_tweets = df_tweets.dropna(subset=['text']).reset_index(drop=True)

In [10]:
#Change source solumn to string data type
df_tweets["source"] = df_tweets["source"].apply(str)

In [11]:
#Change source column into all lower case
df_tweets["source"] = df_tweets["source"].str.lower()

In [12]:
#Remove any tweets with source column value containing the word "bot"
df_tweets= df_tweets[~df_tweets["source"].str.contains("bot")]

In [13]:
#Restructure the dataframe
df_tweets = df_tweets.loc[:,["date","text","user_followers","user_friends","user_favourites"]]

In [14]:
#Remove rows with date column values matching below
df_tweets = df_tweets[~df_tweets["date"].str.contains('YieldFarming' or  'Airdrop' or  'Binance' or  'Bitcoin' or  'pancakeswap' or  'BNB' or  'cryptocurrency' or  'DeFi' or  'BTC' or  'BinanceSmartChain' or  'BSC' or  'pufferswap' or  'DeFi' or  'bsc' or  'bnb' or  'bitcoin' or  'cryto' or  'Airdrop' or  'Airdrop' or 'btc')]

In [15]:
#Change date column to datetime data type
df_tweets['date'] = pd.to_datetime(df_tweets['date'] , format='%Y-%m-%d %H:%M:%S', errors='coerce')

In [16]:
#Remove any tweets with date columns before '2021-7-4'
df_tweets = df_tweets[df_tweets['date']>=pd.to_datetime('2021-7-4')]

In [17]:
#Change text column to string data type
df_tweets["text"] = df_tweets["text"].apply(str)

In [18]:
#Save the text column values into a list
tweets = df_tweets["text"].tolist()

In [19]:
#Use the tweets list to clean up all tweets, removing any extra characters
def processTweet(tweet): #start process_tweet
    # process the tweets
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)#trim
    tweet = tweet.strip('\'"')
    return tweet
#end
tweets_cleaned = []
#Read the tweets one by one and process it
for i in range (0,len(tweets)):
    processedTweet = processTweet(tweets[i])
    tweets_cleaned.append(processedTweet)
print(len(tweets_cleaned))
tweets_cleaned[0:11]

3413412


['why ‘cardano is just teasing us’ 💡 URL cardano ada lovelace bitcoin cardanocommunity adalovelace altcoin btc blockchain $ada cardanonewsfeed crypto URL',
 'thailand boutiquehotels for sale! willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency URL',
 'lunaland ($lln) will burn coin supply until we reach 1 lunar distance in coins. (400m coins) a burn rate of 99.96% from the initial pool of 1 trillion coins reduces coin supply, presale blockchain cryptotrading investing cryptocurrency bitcoin trading altcoin cryptonews',
 'ilovebitcoin because people never assume that if you do bitcoin, you can fix all of their computers',
 '$bsc $btc bnb bitcoin bsc earnbnb btc URL',
 'crypto pump alert! - moma protocol (ico) starts july 6, 2021 at 01:00am - join the pump here🤑: URL in 30 mins! wsb doge crypto cryptocurrency btc bitcoi

In [20]:
#Import and use vaderSentiment to calculate polarity scores of tweets
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [21]:
vader = SentimentIntensityAnalyzer()
def getCompoundScore(Tweets):
    score = vader.polarity_scores(Tweets)['compound']
    return score
scores = []
#Read the tweets one by one and get the sentiment score
for i in range (0,len(tweets_cleaned)):
    tweets_score = getCompoundScore(tweets_cleaned[i])
    scores.append(tweets_score) 

In [22]:
#store the tweet with polarity score in a dictionary
df ={"tweets": tweets_cleaned, "score" : scores}
data = pd.DataFrame(df)#transfer into a dataframe
#print(data.shape)
#data.head(11)

In [23]:
merged_tweets = df_tweets.copy()

In [24]:
#Merge the tweets with polarity score into a new dataframe
df_tweets.reset_index(drop=True, inplace=True)
data.reset_index(drop=True, inplace=True)
merged_tweets = pd.concat([df_tweets,data],axis=1).reset_index(drop=True)
merged_tweets = merged_tweets.drop('text',axis=1)
#print(merged_tweets.shape)
#merged_tweets.head(10)

In [25]:
#Sort the dataframe by date
merged_tweets = merged_tweets.sort_values(by='date')

In [26]:
merged_tweets

,date,user_followers,user_friends,user_favourites,tweets,score
46987,2021-07-04 00:00:00,32.0,19.0,82.0,bitcoin current price: $ 34682.16 € 29214.49 c...,0.0000
46986,2021-07-04 00:00:01,163.0,1.0,0.0,bitcoin: $34682.84 💚 +99.16 last 1 hour (+0.29...,0.9274
46985,2021-07-04 00:00:02,610.0,1103.0,6270.0,exposed: congressman trying to ‘shut down’ cry...,-0.0772
46984,2021-07-04 00:00:02,55401.0,60571.0,77.0,6:00 pm &gt;&gt; $btc price: $34673.75000000 &...,0.0000
46982,2021-07-04 00:00:03,310.0,114.0,943.0,dogecoin (doge) failed to cross the 100 dma! U...,-0.5562
...,...,...,...,...,...,...
3392394,2022-07-03 23:59:48,1239.0,430.0,3563.0,victims of luna are creating a cluna nft colle...,-0.0258
3392393,2022-07-03 23:59:50,40.0,105.0,20.0,AT_USER bitcoin fixes this,0.0000
3392392,2022-07-03 23:59:52,2419.0,4.0,18.0,bitcoin last price $19318 btc 🚀 daily indicato...,0.6369
3392391,2022-07-03 23:59:55,550.0,91.0,9.0,orion money (orion) went up 11.7 percent in th...,0.0000


In [27]:
#Create a function that categorizes the sentiment score into Positive, Negative or Neutral
def getSentiment(score):
    if score < 0:
        return "negative"
    elif score == 0:
        return "neutral"
    else:
        return "positive"

In [28]:
#Create a sentiment column and use the getSentiment function to fill values
merged_tweets['sentiment'] = merged_tweets['score'].apply(getSentiment)

In [29]:
df_btc

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,TB Base Volume,TB Quote Volume,Ignore
0,2019-07-30 00:00:00,9509.07,9535.00,9503.07,9517.75,381.293856,2019-07-30 00:29:59.999000064,3.631211e+06,3492,216.528742,2.062189e+06,0
1,2019-07-30 00:30:00,9517.03,9539.00,9507.00,9532.58,258.514869,2019-07-30 00:59:59.999000064,2.462134e+06,3133,141.887550,1.351182e+06,0
2,2019-07-30 01:00:00,9533.59,9534.00,9500.00,9500.25,275.797270,2019-07-30 01:29:59.999000064,2.624322e+06,3583,114.238457,1.087402e+06,0
3,2019-07-30 01:30:00,9500.25,9514.46,9460.71,9465.25,485.218470,2019-07-30 01:59:59.999000064,4.604004e+06,5330,225.681291,2.141459e+06,0
4,2019-07-30 02:00:00,9465.25,9488.94,9430.01,9460.42,498.669626,2019-07-30 02:29:59.999000064,4.717269e+06,6197,228.535910,2.162163e+06,0
...,...,...,...,...,...,...,...,...,...,...,...,...
51306,2022-07-04 20:30:00,19783.48,19812.19,19747.34,19782.79,692.590040,2022-07-04 20:59:59.999000064,1.369969e+07,13963,374.688800,7.410945e+06,0
51307,2022-07-04 21:00:00,19782.79,19823.97,19749.51,19781.71,637.498450,2022-07-04 21:29:59.999000064,1.261613e+07,11000,325.724110,6.446203e+06,0
51308,2022-07-04 21:30:00,19781.71,20031.58,19777.66,19958.75,1298.124410,2022-07-04 21:59:59.999000064,2.587612e+07,26022,763.196040,1.521461e+07,0
51309,2022-07-04 22:00:00,19958.76,19981.87,19871.88,19956.60,1260.138300,2022-07-04 22:29:59.999000064,2.509992e+07,19054,636.915190,1.268486e+07,0


In [30]:
#Create a date column that stores only the value
df_btc['date'] = df_btc['Open Time'].dt.date

In [31]:
#Create of the bitcoin price history dataset
newdf = df_btc.copy()

In [32]:
#Trim dataset for price history from '2021-7-4' to '2022-7-3'
newdf = newdf[newdf['date']>=pd.to_datetime('2021-7-4')]

c:\Users\Mo\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


In [33]:
#Convert date column to datetime data type
newdf['date'] = pd.to_datetime(newdf['date'])

In [34]:
#Setting the date column as the index
newdf = newdf.set_index('date')

In [35]:
#Resampling the 30minute interval price history dataset to a daily average price history dataset
newdf = newdf.resample('D').mean()

In [36]:
newdf.head()

,Open,High,Low,Close,Volume,Quote Asset Volume,Number of Trades,TB Base Volume,TB Quote Volume,Ignore
date,,,,,,,,,,
2021-07-04,35269.082708,35370.196667,35180.610625,35281.847292,910.489079,3.217621e+07,24270.208333,441.256981,1.558724e+07,0.0
2021-07-05,34095.084375,34208.018125,33943.144792,34061.738125,1335.914047,4.548763e+07,23111.812500,656.038654,2.233265e+07,0.0
2021-07-06,34157.270208,34297.825833,34007.968333,34168.104375,1212.720757,4.149356e+07,24680.979167,579.207385,1.981810e+07,0.0
2021-07-07,34553.322708,34646.623750,34431.268542,34545.995833,1120.990035,3.870121e+07,21696.166667,568.644248,1.962468e+07,0.0
2021-07-08,32895.774583,33017.407083,32747.872292,32875.238542,1461.176673,4.802027e+07,26206.083333,731.346984,2.401934e+07,0.0


In [37]:
#Creating a new column that uses the Open column to calculate rate of change of Opening price
newdf['roc'] = newdf['Open'].pct_change()

In [38]:
merged_tweets

,date,user_followers,user_friends,user_favourites,tweets,score,sentiment
46987,2021-07-04 00:00:00,32.0,19.0,82.0,bitcoin current price: $ 34682.16 € 29214.49 c...,0.0000,neutral
46986,2021-07-04 00:00:01,163.0,1.0,0.0,bitcoin: $34682.84 💚 +99.16 last 1 hour (+0.29...,0.9274,positive
46985,2021-07-04 00:00:02,610.0,1103.0,6270.0,exposed: congressman trying to ‘shut down’ cry...,-0.0772,negative
46984,2021-07-04 00:00:02,55401.0,60571.0,77.0,6:00 pm &gt;&gt; $btc price: $34673.75000000 &...,0.0000,neutral
46982,2021-07-04 00:00:03,310.0,114.0,943.0,dogecoin (doge) failed to cross the 100 dma! U...,-0.5562,negative
...,...,...,...,...,...,...,...
3392394,2022-07-03 23:59:48,1239.0,430.0,3563.0,victims of luna are creating a cluna nft colle...,-0.0258,negative
3392393,2022-07-03 23:59:50,40.0,105.0,20.0,AT_USER bitcoin fixes this,0.0000,neutral
3392392,2022-07-03 23:59:52,2419.0,4.0,18.0,bitcoin last price $19318 btc 🚀 daily indicato...,0.6369,positive
3392391,2022-07-03 23:59:55,550.0,91.0,9.0,orion money (orion) went up 11.7 percent in th...,0.0000,neutral


In [39]:
#Create a new column named timestamp which is a copy of the date column
merged_tweets['timestamp'] = merged_tweets['date']

In [40]:
#Change the date field to a date(without timestamp) field to match the merged_tweets date column
merged_tweets['date'] = merged_tweets['date'].dt.date

In [41]:
#Reset the index of the dataframe
newdf = newdf.reset_index()

In [42]:
#Convert the date column into a datetime data type to match newdf
merged_tweets['date'] = pd.to_datetime(merged_tweets['date'])

In [43]:
#Creating a merged dataframe containing tweets and their daily price and rate of change
finaldf = pd.merge(merged_tweets,newdf)

In [44]:
finaldf

,date,user_followers,user_friends,user_favourites,tweets,score,sentiment,timestamp,Open,High,Low,Close,Volume,Quote Asset Volume,Number of Trades,TB Base Volume,TB Quote Volume,Ignore,roc
0,2021-07-04,32.0,19.0,82.0,bitcoin current price: $ 34682.16 € 29214.49 c...,0.0000,neutral,2021-07-04 00:00:00,35269.082708,35370.196667,35180.610625,35281.847292,910.489079,3.217621e+07,24270.208333,441.256981,1.558724e+07,0.0,NaN
1,2021-07-04,163.0,1.0,0.0,bitcoin: $34682.84 💚 +99.16 last 1 hour (+0.29...,0.9274,positive,2021-07-04 00:00:01,35269.082708,35370.196667,35180.610625,35281.847292,910.489079,3.217621e+07,24270.208333,441.256981,1.558724e+07,0.0,NaN
2,2021-07-04,610.0,1103.0,6270.0,exposed: congressman trying to ‘shut down’ cry...,-0.0772,negative,2021-07-04 00:00:02,35269.082708,35370.196667,35180.610625,35281.847292,910.489079,3.217621e+07,24270.208333,441.256981,1.558724e+07,0.0,NaN
3,2021-07-04,55401.0,60571.0,77.0,6:00 pm &gt;&gt; $btc price: $34673.75000000 &...,0.0000,neutral,2021-07-04 00:00:02,35269.082708,35370.196667,35180.610625,35281.847292,910.489079,3.217621e+07,24270.208333,441.256981,1.558724e+07,0.0,NaN
4,2021-07-04,310.0,114.0,943.0,dogecoin (doge) failed to cross the 100 dma! U...,-0.5562,negative,2021-07-04 00:00:03,35269.082708,35370.196667,35180.610625,35281.847292,910.489079,3.217621e+07,24270.208333,441.256981,1.558724e+07,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3413407,2022-07-03,1239.0,430.0,3563.0,victims of luna are creating a cluna nft colle...,-0.0258,negative,2022-07-03 23:59:48,19179.100208,19233.236458,19124.494375,19180.412500,1064.322215,2.042200e+07,18349.708333,528.374228,1.014068e+07,0.0,-0.003195
3413408,2022-07-03,40.0,105.0,20.0,AT_USER bitcoin fixes this,0.0000,neutral,2022-07-03 23:59:50,19179.100208,19233.236458,19124.494375,19180.412500,1064.322215,2.042200e+07,18349.708333,528.374228,1.014068e+07,0.0,-0.003195
3413409,2022-07-03,2419.0,4.0,18.0,bitcoin last price $19318 btc 🚀 daily indicato...,0.6369,positive,2022-07-03 23:59:52,19179.100208,19233.236458,19124.494375,19180.412500,1064.322215,2.042200e+07,18349.708333,528.374228,1.014068e+07,0.0,-0.003195
3413410,2022-07-03,550.0,91.0,9.0,orion money (orion) went up 11.7 percent in th...,0.0000,neutral,2022-07-03 23:59:55,19179.100208,19233.236458,19124.494375,19180.412500,1064.322215,2.042200e+07,18349.708333,528.374228,1.014068e+07,0.0,-0.003195


In [45]:
finaldf.to_csv('mergeddf070922.csv')